In [2]:
import xarray as xr
ds = xr.open_dataset("C:/STEMMUS_SCOPE_data/global/landcover/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2014-v2.0.7cds.nc")

In [5]:
ds.isel(lat=0, lon=0).load()

<xarray.Dataset>
Dimensions:              (time: 1, bounds: 2)
Coordinates:
    lat                  float64 90.0
    lon                  float64 -180.0
  * time                 (time) datetime64[ns] 2014-01-01
Dimensions without coordinates: bounds
Data variables:
    lccs_class           (time) uint8 210
    processed_flag       (time) float32 0.0
    current_pixel_state  (time) float32 0.0
    observation_count    (time) uint16 0
    change_count         (time) uint8 0
    crs                  int32 -2147483647
    lat_bounds           (bounds) float64 90.0 90.0
    lon_bounds           (bounds) float64 -180.0 -180.0
    time_bounds          (time, bounds) datetime64[ns] 2014-01-01 2014-12-31
Attributes: (12/38)
    id:                         ESACCI-LC-L4-LCCS-Map-300m-P1Y-2014-v2.0.7cds
    title:                      Land Cover Map of ESA CCI brokered by CDS
    summary:                    This dataset characterizes the land cover of ...
    type:                       ESACCI-LC-L4-LCCS-Map-300m-P1Y
    project:                    Climate Change Initiative - European Space Ag...
    references:                 http://www.esa-landcover-cci.org/
    ...                         ...
    geospatial_lon_max:         180
    spatial_resolution:         300m
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.002778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.002778

In [18]:
ds.sel(lat=slice(54, 50), lon=slice(3, 6)).lccs_class.attrs["flag_values"]

array([  0,  10,  11,  12,  20,  30,  40,  50,  60,  61,  62,  70,  71,
        72,  80,  81,  82,  90, 100, 110, 120, 121, 122, 130, 140, 150,
       151, 152, 153, 160, 170, 180, 190, 200, 201, 202, 210, 220],
      dtype=uint8)

In [20]:
def get_landcover_table(ds):
    """Get the lookup table to convert the flag values to a landcover."""
    flag_meanings = ds["lccs_class"].attrs["flag_meanings"].split(" ")
    flag_values = ds["lccs_class"].attrs["flag_values"]
    return {key: flag for key, flag in zip(flag_values, flag_meanings)}

translation_table = get_landcover_table(ds)

In [ ]:
lat=52, lon=4


In [56]:
import numpy as np
lat = 52.0540
lon = 6.0242
lat_idx = np.logical_and(ds.lat_bounds.isel(bounds=0) >= lat, ds.lat_bounds.isel(bounds=1) < lat).argmax()
lon_idx = np.logical_and(ds.lon_bounds.isel(bounds=0) <= lon, ds.lon_bounds.isel(bounds=1) > lon).argmax()

In [58]:
landcover_val = ds.isel(lat=lat_idx, lon=lon_idx)["lccs_class"].to_numpy()[0]
translation_table[landcover_val]

'mosaic_tree_and_shrub'

In [66]:
import pandas as pd
df = pd.read_csv("../lccs_to_igbp_table.csv", index_col="lccs_class")


In [70]:
igbp_veg_long = df.loc[landcover_val].to_numpy()[0]
igbp_veg_long

'Woody savannas'